In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import imutils
import time
import uuid
import requests
import json

In [2]:
URL = "http://localhost:5000"
ORG_NAME = "org1"
USER_NAME = "User1"

CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]
CLASS_LABELS_EMOJIS = ["👿", "🤢" , "😱" , "😊" , "😐 ", "😔" , "😲" ]

font = cv2.FONT_HERSHEY_SIMPLEX  
origin = (10, 10)
fontScale = 1
color = (255, 0, 0)
thickness = 2

In [3]:
model = tf.keras.models.load_model("Bin/EmoDetection.h5")
face_cascade = cv2.CascadeClassifier('Bin/haarcascade_frontalface_default.xml')

In [4]:
def getEmotion(img) : 
    img = cv2.resize(img, (48, 48))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.stack((img,)*3, axis=-1)
    img = tf.keras.applications.densenet.preprocess_input(img)
    img = img/255.0
    res = model.predict(img.reshape(1, 48, 48, 3),verbose = 0)[0]
    id = np.argmax(res)
    return {
        "class_id" : id,
        "class_name" : CLASS_LABELS[id],
        "class_emoji" : CLASS_LABELS_EMOJIS[id],
        "dist" : res
    }

def getEmotionFromUrl(url) : 
  return getEmotion(imutils.url_to_image(url))

def createSession(session) : 
    payload = json.dumps({
      "org": ORG_NAME,
      "name": USER_NAME,
      "session": session
    })
    headers = {
      'Content-Type': 'application/json'
    }
    requests.request("POST", URL+"/createSession", headers=headers, data=payload)

def logData(session, data) : 
    payload = json.dumps({
      "session": session,
      "log": data
    })
    headers = {
      'Content-Type': 'application/json'
    }
    requests.request("POST", URL+"/log", headers=headers, data=payload)

In [5]:
vid = cv2.VideoCapture(0)
session = str(uuid.uuid4())
createSession(session)
session_emos = []

try :
    count = -1
    while(True):
        count += 1
        ret, frame = vid.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0 : 
            x,y,w,h = faces[0]
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            img = frame[y:y+h, x:x+w]
            if count%10==0 : 
                data = getEmotion(img)
                session_emos.append(data["class_emoji"])
                print(data["class_emoji"], end="")
                logData(session, {
                    "class" : int(data["class_id"]),
                    "dist" : [float(i) for i in data["dist"]]
                })
                img =  cv2.putText(img, data["class_name"], origin, font, fontScale, color, thickness, cv2.LINE_AA)
                cv2.imshow('Face', img)
                count = 0
        cv2.imshow('Camera', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(0.1)
        
except Exception as e: 
    print(e)
finally : 
    vid.release()
    cv2.destroyAllWindows()

😔😊😐 😊😊😊👿😊😊😐 😊😊😊😊😊😐 😱😱😔😱😐 😱😱😔😱😱😱👿😐 😊😐 😊😐 😊😱😱😔😊😊😊